In [99]:
from api_key import get_api_key_jef, get_api_key_sha, get_api_key_sup
import google.generativeai as genai
from vpn_control import windscribe
import time
import re
import pandas as pd
import numpy as np



In [100]:
windscribe("connect", "Atlanta")
time.sleep(5)

In [101]:
passw = "2002"
LLM_MODEL = "gemini-1.5-flash" #gemini-1.5-pro-latest

In [102]:
def setup_prompt_env(passw, extra_instruct):


  genai.configure(api_key=get_api_key_sup(passw))

  # Set up the model
  generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 2500,
  }

  safety_settings = [
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
  ]

  system_instruction = f"""You must modify all the wildcards <*> with varibles which would make sense in that conxtext such as time, IP address etc. Additonally {extra_instruct}. Pad each log with a begin and end token <B> <E>. For each input create 25 logs with different varibles filled into the wildcards <*>. Find examples below
  
  INPUT: PacketResponder <*> for block blk_<*> terminating
  OUTPUT: 
    [1] <B>PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating<E>
    [2] <B>PacketResponder: PacketResponder 0 for block blk_-6952295868487656571 terminating<E>
    [3] <B>PacketResponder: PacketResponder 2 for block blk_8229193803249955061 terminating<E>
    [4] <B>PacketResponder: PacketResponder 2 for block blk_-6670958622368987959 terminating<E>
    [5] <B>PacketResponder: PacketResponder 1 for block blk_5017373558217225674 terminating<E>
    ...
    [25] <B>PacketResponder: PacketResponder 1 for block blk_509586258217225674 terminating<E>

  
 Be creative with numbers DO NOT use repeating or ordered numbers such as 111 or 1234. Do NOT change the rest of the long, only <*> and do not give extra text."""

  model = genai.GenerativeModel(model_name=LLM_MODEL,
                                generation_config=generation_config,
                                system_instruction=system_instruction,
                                safety_settings=safety_settings)

  convo = model.start_chat(history=[
  ])

  return convo


In [103]:
def create_logs_LLM(message, convo, extra_instruct):
    message =  message 
    convo.send_message(message)
    suggestion = convo.last.text
    print(suggestion)
    return suggestion

In [104]:

def setup_LLM_logs(passw, extra_instruct, message):
    convo_create = setup_prompt_env(passw, extra_instruct)
    res = create_logs_LLM(message, convo_create, extra_instruct)
    return res



In [105]:
setup_LLM_logs(passw, ".", "BLOCK* NameSystem.addStoredBlock: blockMap updated: <*>:<*> is added to blk_<*> size <*>")
# #BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.43.115:50010 is added to blk_3050920587428079149 size 67108864

<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.10:50010 is added to blk_8229193803249955061 size 1024<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.1:50020 is added to blk_38865049064139660 size 2048<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 172.17.17.1:50030 is added to blk_-6952295868487656571 size 4096<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.1:50040 is added to blk_5017373558217225674 size 8192<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.2:50050 is added to blk_-6670958622368987959 size 16384<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 172.17.17.2:50060 is added to blk_509586258217225674 size 32768<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.2:50070 is added to blk_8229193803249955062 size 65536<E>
<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.3:50080 is added to blk_38865049064139661 size 131072<E>
<B>BLOCK* NameSystem.addStored

'<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.10:50010 is added to blk_8229193803249955061 size 1024<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.1:50020 is added to blk_38865049064139660 size 2048<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 172.17.17.1:50030 is added to blk_-6952295868487656571 size 4096<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.1:50040 is added to blk_5017373558217225674 size 8192<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.2:50050 is added to blk_-6670958622368987959 size 16384<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 172.17.17.2:50060 is added to blk_509586258217225674 size 32768<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 192.168.1.2:50070 is added to blk_8229193803249955062 size 65536<E>\n<B>BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.0.0.3:50080 is added to blk_38865049064139661 size 131072<E>\n<B>BLOCK* NameSystem.

In [106]:
# setup_LLM_logs(passw, ".", "Received block blk_<*> of size <*> from /<*>")
# #Received block blk_3587508140051953248 of size 67108864 from /10.251.42.84

In [107]:
#create func that checks only <*> has been replaced

In [108]:
# append
# 081109 203615 148 INFO dfs.DataNode$ 
# 081109 203807 222 INFO dfs.DataNode$
# 081109 204132 26 INFO dfs.FSNamesystem:
#081109 204655 556 INFO dfs.DataNode$PacketResponder: 

In [109]:
windscribe("disconnect")